In [1]:
import requests
import pandas as pd
import plotly.express as px

# -----------------------------
# Fetch COVID Case Data
# -----------------------------
cases_url = "https://disease.sh/v3/covid-19/countries"
cases_response = requests.get(cases_url)
cases_data = cases_response.json()

cases_df = pd.DataFrame(cases_data)

# Select important columns
cases_df = cases_df[[
    'country', 'cases', 'todayCases',
    'deaths', 'todayDeaths',
    'recovered', 'active', 'population'
]]

# -----------------------------
# Fetch Vaccination Data
# -----------------------------
vaccine_url = "https://disease.sh/v3/covid-19/vaccine/coverage/countries?lastdays=1"
vaccine_response = requests.get(vaccine_url)
vaccine_data = vaccine_response.json()

vacc_list = []
for item in vaccine_data:
    country = item['country']
    vaccines = list(item['timeline'].values())[0]
    vacc_list.append([country, vaccines])

vaccine_df = pd.DataFrame(vacc_list, columns=['country', 'vaccinations'])

# -----------------------------
# Merge Data
# -----------------------------
final_df = pd.merge(cases_df, vaccine_df, on='country', how='left')

# -----------------------------
# Visualization 1: Top 10 Countries by Cases
# -----------------------------
top_cases = final_df.sort_values(by='cases', ascending=False).head(10)

fig1 = px.bar(
    top_cases,
    x='country',
    y='cases',
    title='Top 10 Countries by COVID-19 Cases',
    labels={'cases': 'Total Cases'}
)
fig1.show()

# -----------------------------
# Visualization 2: Vaccination vs Cases
# -----------------------------
fig2 = px.scatter(
    final_df,
    x='vaccinations',
    y='cases',
    size='population',
    hover_name='country',
    title='COVID-19 Cases vs Vaccinations',
    labels={
        'vaccinations': 'Total Vaccinations',
        'cases': 'Total Cases'
    }
)
fig2.show()

# -----------------------------
# Visualization 3: Active Cases by Country (Top 10)
# -----------------------------
top_active = final_df.sort_values(by='active', ascending=False).head(10)

fig3 = px.pie(
    top_active,
    names='country',
    values='active',
    title='Active COVID-19 Cases Distribution'
)
fig3.show()

print("COVID-19 Data Analysis Completed Successfully!")

COVID-19 Data Analysis Completed Successfully!
